## Посмотрим на данные

Проверим, какие столбцы в них присутствуют, сколько классов и сколько всего объектов

In [23]:
import os
if os.getcwd().endswith('lab01_knn'):
    os.chdir('..')

In [24]:
from lab01_knn.knn import KNNClassifier, KNNRegressionClassifier
from lab01_knn.common.metrics import confusion_matrix, micro_f1_score, macro_f1_score
from lab01_knn.common.data import k_fold_validation, one_out_validation, normalize, one_hot_encode

In [25]:
import pandas as pd
import numpy as np

In [26]:
data = pd.read_csv('lab01_knn/resources/heart-h.csv')
data

,V1,V2,V3,V4,V5,V6,V7,Class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [27]:
data['Class'].unique()

array([1, 2, 3], dtype=int64)

## Нормализуем столбцы (для начала можно все)

Параллельно заменим класс 5 на класс 0, чтобы не сбивать нумерацию

In [29]:
data.at[data['Class'] == 5, 'Class'] = 0
data = normalize(data, columns=list(map(lambda n: f'V{n}', range(1, 14))))
data

1: 0.8963163633751868/0.9032384463545415
2: 0.8963163633751868/0.9032384463545415
3: 0.8786112906701142/0.8855611933540375
4: 0.8866391941391942/0.8920747014834418
5: 0.8931684981684981/0.8982024390372667
6: 0.8916544566544566/0.8965004633382904
7: 0.8924297924297925/0.8962011245233332
8: 0.9058974358974361/0.9091493863781063
9: 0.9107722832722832/0.9145835510152125
10: 0.9104029304029304/0.9152442733008239
11: 0.9059525605113838/0.9118107562246021
12: 0.9055862601450837/0.9114747661948861
13: 0.9052871148459385/0.9101364090441704
14: 0.900848775407599/0.9070517870388108
15: 0.9056654456654456/0.9108212941447487
16: 0.9065201465201467/0.9118007409822191
17: 0.9063705738705738/0.9114864867004885
18: 0.9017826617826618/0.9077260443398089
19: 0.9111324786324788/0.9162484549316078


In [ ]:
X = data.drop('Class', axis=1).values
y = data['Class'].values

## Проверим обычный kNN-классификатор на некоторых k

In [ ]:
for k in [1, 2, 4, 8, 16, 32, 64, 128]:
    scores_micro = k_fold_validation(X, y, KNNClassifier(k=k), lambda y, ypred: micro_f1_score(confusion_matrix(y, ypred, classes=5)))
    scores_macro = k_fold_validation(X, y, KNNClassifier(k=k), lambda y, ypred: macro_f1_score(confusion_matrix(y, ypred, classes=5)))
    print(f'{k}: {np.mean(scores_micro)}/{np.mean(scores_macro)}')

# Теперь попробуем провести регрессию

Для нормализации будем брать только не-категориальные столбцы

In [ ]:
params = {
    'distance': ['euclidean', 'manhattan', 'chebyshev'],
    'kernel': ['uniform', 'triangular', 'epanechnikov', 'quartic', 'triweight', 'tricube', 'gaussian', 'cosine', 'logistic', 'sigmoid'],
    'window': ['fixed', 'variable'],
    'k': [1, 5, 10, 25, 50, 100, 150]
}

data = pd.read_csv('lab01_knn/resources/heart-h.csv')
data.at[data['Class'] == 5, 'Class'] = 0
data = normalize(data, columns=['V1', 'V4', 'V5', 'V8', 'V10'])

In [ ]:
list(map(pd.Series.unique, [data['V11'], data['V12'], data['V13']]))

У данных нет описания, но эти признаки выглядят больше категориальными, чем численными, хотя странно, что их тогда так записали... Можно пока попробовать оставить как есть и нормализовать

In [ ]:
data = normalize(data, columns=['V11', 'V12', 'V13'])
data

In [ ]:
X = data.drop('Class', axis=1).values
y = data['Class'].values

In [ ]:
best_result = None
best_score = 0

def scoring(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, classes=5)
    return (micro_f1_score(cm) + macro_f1_score(cm)) / 2

cnt = 1
for value in params.values():
    cnt *= len(value)
print(f'Options: {cnt}')

In [ ]:
finished = 0

for distance in params['distance']:
    for kernel in params['kernel']:
        for window in params['window']:
            for k in params['k']:
                model = KNNRegressionClassifier(k=k, distance=distance, kernel=kernel, window=window, h=k)
                score = one_out_validation(X, y, model, scoring)
                if score > best_score:
                    best_result = {'distance': distance, 'kernel': kernel, 'window': window, 'k': k}
                    best_score = score
                finished += 1
                if finished % 42 == 0:
                    print(f'{finished // 42 * 10}%')

print(best_result)
print(best_score)

Для полученных параметров можно построить графики в зависимости от k, но лучше это сделать позже, чтобы сравнить с OHE и без него.

## Теперь можно попробовать добавить OneHotEncoding

Параллельно можно так же закодировать другие категориальные признаки

In [ ]:
data1 = data.copy(deep=True)

data = pd.read_csv('lab01_knn/resources/heart-h.csv')
data.at[data['Class'] == 5, 'Class'] = 0
data = normalize(data, columns=['V1', 'V4', 'V5', 'V7', 'V8', 'V10'])

data2 = one_hot_encode(data, columns=['V2', 'V3', 'V4', 'V6', 'V9', 'V11', 'V12', 'V13', 'Class'])

In [ ]:
data2

In [ ]:
classes = ['Class@0', 'Class@1', 'Class@2', 'Class@3', 'Class@4']
X2 = data2.drop(columns=classes).values
y2 = data2.loc[:, classes].values

In [ ]:
def custom_eval(model):
    predictions = np.zeros(y2.shape, dtype=float)
    for i in range(X2.shape[0]):
        X_train = np.vstack((X2[:i], X2[i + 1:]))
        for j in range(5):
            y_train = np.hstack((y2[:i, j], y2[i + 1:, j]))
            
            y_pred = model.clone_unfit().fit(X_train, y_train).predict(np.asarray([X2[i]]))
            predictions[i][j] = y_pred[0]
    return np.argmax(predictions, axis=1)

# test
print(scoring(data['Class'].values, custom_eval(KNNRegressionClassifier(k=150, kernel='gaussian', window='variable'))))

Имеет смысл уменьшить перебор, потому что времени будет уходить в 5 раз дольше..

In [ ]:
params = {
    'distance': ['euclidean', 'manhattan'],
    'kernel': ['triangular', 'epanechnikov', 'triweight', 'gaussian', 'logistic', 'sigmoid'],
    'window': ['variable'],
    'k': [1, 2, 5, 8, 15, 25, 50]
}

cnt = 1
for value in params.values():
    cnt *= len(value)
print(f'Options: {cnt * 5}')

In [ ]:
best_result2 = None
best_score2 = 0
finished2 = 0

for distance in params['distance']:
    for kernel in params['kernel']:
        for window in params['window']:
            for k in params['k']:
                model = KNNRegressionClassifier(k=k, distance=distance, kernel=kernel, window=window, h=k)
                score = scoring(data['Class'].values, custom_eval(model))
                if score > best_score2:
                    best_result2 = {'distance': distance, 'kernel': kernel, 'window': window, 'k': k}
                    best_score2 = score
                finished2 += 5
                if finished2 % 42 < 5:
                    print(f'{finished2 // 42 * 10}%')

print(best_result2)
print(best_score2)

Мы видим, что в обоих случаях лучший вариант получается при gaussian ядре, так что можно попробовать построить графики зависимости f1_score от k в двух случаях:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

ks = [1, 2, 5, 8, 15, 25, 40, 65, 100, 140, 185]

for it, distance in enumerate(['euclidean', 'manhattan']):
    scores1, scores2 = [], []
    for k in ks:
        model = KNNRegressionClassifier(k=k, distance=distance, kernel='gaussian', window='variable', h=k)
        scores1.append(one_out_validation(X, y, model, scoring))
        scores2.append(scoring(data['Class'].values, custom_eval(model)))
    plt.subplot(121 + it)
    plt.plot(ks, scores1)
    plt.plot(ks, scores2, color='red')

## Наблюдения

Почему-то с OHE результаты получаются не лучше, чем без него. Но, видимо, это специфика этого датасета (как минимум отчасти), потому что оптимальные результаты, полученные каждым методом, дают примерно одинаковые оценки качества :(